In [1]:
using PythonCall
using JLD2
using JLD2: Group
using OceananigansLagrangianFilter
using Oceananigans.Fields: Center
using Oceananigans.BoundaryConditions: PeriodicBoundaryCondition
using ProgressBars

Precompiling packages...
   6979.0 ms  ✓ OceananigansLagrangianFilter
  1 dependency successfully precompiled in 9 seconds. 504 already precompiled.


In [1]:
using PythonCall
PyLinearNDInterpolator() = pyimport("scipy.interpolate").LinearNDInterpolator()


PyLinearNDInterpolator (generic function with 1 method)

In [2]:
points = Array([[0,0] [0,1] [0,2] [1,0] [1,1] [1,2] [2,0] [2,1] [2,2]])
values = Array([1,1,1,2,2,2,3,3,3])
interpolator = PyLinearNDInterpolator(points', values)
new_points = Array([ [0.5,0.5] [1.5,2] [2,1.5] ])
x_new = Array(range(0, stop=2, length=50)) # Array of x coordinates
y_new = Array(range(0, stop=2, length=60))# Array of y coordinates
x_mesh = reshape(x_new,length(x_new),1)
y_mesh = reshape(y_new,1,length(y_new))
coord_mesh = [x_mesh, y_mesh]
z_interpolated = interpolator(coord_mesh...)
# interpolator(new_points')
z_interpolated.shape

LoadError: MethodError: no method matching PyLinearNDInterpolator(::LinearAlgebra.Adjoint{Int64, Matrix{Int64}}, ::Vector{Int64})

In [4]:
pyconvert(Array, z_interpolated)

50×60 Matrix{Float64}:
 1.0      1.0      1.0      1.0      …  1.0      1.0      1.0      1.0
 1.04082  1.04082  1.04082  1.04082     1.04082  1.04082  1.04082  1.04082
 1.08163  1.08163  1.08163  1.08163     1.08163  1.08163  1.08163  1.08163
 1.12245  1.12245  1.12245  1.12245     1.12245  1.12245  1.12245  1.12245
 1.16327  1.16327  1.16327  1.16327     1.16327  1.16327  1.16327  1.16327
 1.20408  1.20408  1.20408  1.20408  …  1.20408  1.20408  1.20408  1.20408
 1.2449   1.2449   1.2449   1.2449      1.2449   1.2449   1.2449   1.2449
 1.28571  1.28571  1.28571  1.28571     1.28571  1.28571  1.28571  1.28571
 1.32653  1.32653  1.32653  1.32653     1.32653  1.32653  1.32653  1.32653
 1.36735  1.36735  1.36735  1.36735     1.36735  1.36735  1.36735  1.36735
 1.40816  1.40816  1.40816  1.40816  …  1.40816  1.40816  1.40816  1.40816
 1.44898  1.44898  1.44898  1.44898     1.44898  1.44898  1.44898  1.44898
 1.4898   1.4898   1.4898   1.4898      1.4898   1.4898   1.4898   1.4898
 ⋮      

In [65]:
combined_output_filename = "../turbulent_SW/combined_LF_freqc_1_with_tracer.jld2"
original_var_names = ["T","ω"]
velocity_names = ["u","v"]
npad = 0

0

In [69]:

jldopen(combined_output_filename,"r+") do file
    iterations = parse.(Int, keys(file["timeseries/t"]))
    grid = file["serialized/grid"]
    coord_dict = _create_coords(grid)
    
    # Work out the periodic directions
    test_var = original_var_names[1]
    BCs = file["timeseries/$test_var/serialized/boundary_conditions"]
    periodic_dimensions = []
    if BCs.west == PeriodicBoundaryCondition()
        push!(periodic_dimensions,"x")
    end
    if BCs.south == PeriodicBoundaryCondition()
        push!(periodic_dimensions,"y")
    end
    if BCs.top == PeriodicBoundaryCondition()
        push!(periodic_dimensions,"z")
    end
    
    # First add the necessary serialized entry for each new variable
    for var in original_var_names 
        new_path = "timeseries/$var"*"_filtered_regrid/serialized"
        if haskey(file, new_path)
            Base.delete!(file, new_path) #incase we already tried to write this
        end
        g = Group(file, new_path)
        for property in keys(file["timeseries/$var/serialized"])
            g[property] = file["timeseries/$var/serialized/$property"]
        end
    end
    
    for iter in ProgressBar(iterations[1:10])
        
        Xi_list = []
        regular_coord_mesh = []
        n_true_dims = 0
        for dim in ("x","y","z")
            if dim in keys(coord_dict) # This limits to only the non-singleton dimensions
                n_true_dims +=1
                push!(regular_coord_mesh, coord_dict["$(dim)_mesh"])
                if (dim == "x") && ("u" in velocity_names)
                    Xi_u = coord_dict["x_mesh"] .+ file["timeseries/xi_u/$iter"]
                    # Lose the halo regions (they don't help with fixed boundaries as they're zero, or with periodic as its repeated information)
                    Xi_u = remove_halos(Xi_u,grid)
                    # move xi points outside of domain into domain
                    if "x" in periodic_dimensions                           
                        Xi_u .-= floor.((Xi_u .- coord_dict["x"][grid.Hx+1])./grid.Lx) .* grid.Lx
                    end
                    push!(Xi_list,vec(Xi_u))


                elseif (dim == "x") && !("u" in velocity_names)
                    Xi_u = coord_dict["x_mesh"]
                    # Lose the halo regions 
                    Xi_u = remove_halos(Xi_u,grid)
                    push!(Xi_list,vec(Xi_u))

                elseif (dim == "y") && ("v" in velocity_names)
                    Xi_v =  coord_dict["y_mesh"] .+ file["timeseries/xi_v/$iter"]
                    # Lose the halo regions 
                    Xi_v = remove_halos(Xi_v,grid)
                    # move xi points outside of domain + halo regions into domain
                    if "y" in periodic_dimensions
                        Xi_v .-= floor.((Xi_v .- coord_dict["y"][grid.Hy+1])./grid.Ly) .* grid.Ly
                    end
                    push!(Xi_list,vec(Xi_v))

                elseif (dim == "y") && !("v" in velocity_names)
                    Xi_v = coord_dict["y_mesh"]
                    # Lose the halo regions 
                    Xi_v = remove_halos(Xi_v,grid)
                    push!(Xi_list,vec(Xi_v))

                elseif (dim == "z") && ("w" in velocity_names)
                    Xi_w = coord_dict["z_mesh"] .+ file["timeseries/xi_w/$iter"]
                    # Lose the halo regions 
                    Xi_w = remove_halos(Xi_w,grid)
                    # move xi points outside of domain + halo regions into domain
                    if "z" in periodic_dimensions
                        Xi_w .-= floor.((Xi_w .- coord_dict["z"][grid.Hz+1])./grid.Lz) .* grid.Lz
                    end
                    push!(Xi_list,vec(Xi_w))

                elseif (dim == "z") && !("w" in velocity_names)
                    Xi_w = coord_dict["z_mesh"] .+ zeros(coord_dict["original_size"])
                    # Lose the halo regions 
                    Xi_w = remove_halos(Xi_w,grid)
                    push!(Xi_list,vec(Xi_w))

                else
                    error("Something's wrong")
                end
            end
        
        end

        # Now we do some padding on the periodic dimensions, introducing new elements to the list near the periodic boundaries
        # First construct a matrix that contains the coordinates and the fields to interpolate
        for var in original_var_names
            var_data = file["timeseries/$var"*"_filtered/$iter"]
            # Lose the halo regions 
            var_data = remove_halos(var_data,grid)

            push!(Xi_list, vec(var_data))
        end

        data_tuple = Tuple(Xi_list)
        data_array = hcat(data_tuple...) # This is a matrix where the rows are the data points and the columns are the coordinates then the variables

        # Then we take the array and repeat rows as necessary to add extra padding data
        
        column_number = 1
        n_columns = size(data_array,2)

        for dim in periodic_dimensions
            if dim == "x"
                max_x = coord_dict["x"][end-grid.Hx]
                min_x = coord_dict["x"][grid.Hx+1] 
                xpad = npad*grid.Δxᶜᵃᵃ
                Xi_x_vec = data_array[:,column_number] 
                mask_max_x = (Xi_x_vec .> max_x - xpad)
                mask_min_x = (Xi_x_vec .< min_x + xpad)
                Xi_x_to_repeat = Xi_x_vec[mask_max_x .| mask_min_x]
                # Remove or add Lx
                Xi_x_to_repeat[(Xi_x_to_repeat .> max_x - xpad)] .-= grid.Lx
                Xi_x_to_repeat[(Xi_x_to_repeat .< min_x + xpad)] .+= grid.Lx

                extra_padding = fill(NaN, (length(Xi_x_to_repeat), n_columns))
                extra_padding[:,column_number] = Xi_x_to_repeat
                # And fill in the rest of the columns with straightforward repeateded data
                for i in 1:n_columns
                    if i != column_number # Don't overwrite the x coordinate
                        extra_padding[:,i] = data_array[mask_max_x .| mask_min_x,i]
                    end
                end
                # Now join it on to the data array
                data_array = vcat(data_array, extra_padding)

                # Move along the rows to the next coordinate
                column_number += 1

            elseif dim == "y"
                max_y = coord_dict["y"][end-grid.Hy]
                min_y = coord_dict["y"][grid.Hy+1]
                ypad = npad*grid.Δyᵃᶜᵃ
                Xi_y_vec = data_array[:,column_number]
                mask_max_y = (Xi_y_vec .> max_y - ypad)
                mask_min_y = (Xi_y_vec .< min_y + ypad)
                Xi_y_to_repeat = Xi_y_vec[mask_max_y .| mask_min_y] 

                # Remove or add Ly
                Xi_y_to_repeat[(Xi_y_to_repeat .> max_y - ypad)] .-= grid.Ly
                Xi_y_to_repeat[(Xi_y_to_repeat .< min_y + ypad)] .+= grid.Ly

                extra_padding = fill(NaN, (length(Xi_y_to_repeat), n_columns))
                extra_padding[:,column_number] = Xi_y_to_repeat
                # And fill in the rest of the columns with straightforward repeateded data
                for i in 1:n_columns
                    if i != column_number # Don't overwrite the y coordinate
                        extra_padding[:,i] = data_array[mask_max_y .| mask_min_y,i]
                    end
                end
                # Now join it on to the data array
                data_array = vcat(data_array, extra_padding)
                # Move along to the next coordinate
                column_number += 1
            elseif dim == "z"
                max_z = coord_dict["z"][end-grid.Hz]
                min_z = coord_dict["z"][grid.Hz+1]
                zpad = npad*grid.Δz.cᵃᵃᶜ
                Xi_z_vec = data_array[:,column_number]
                mask_max_z = ((Xi_z_vec .> max_z - zpad))
                mask_min_z = ((Xi_z_vec .< min_z + zpad))
                Xi_z_to_repeat = Xi_z_vec[mask_max_z .| mask_min_z] 

                # Remove or add Lz
                Xi_z_to_repeat[(Xi_z_to_repeat .> max_z - zpad)] .-= grid.Lz
                Xi_z_to_repeat[(Xi_z_to_repeat .< min_z + zpad)] .+= grid.Lz

                extra_padding = fill(NaN, (length(Xi_z_to_repeat), n_columns))
                extra_padding[:,column_number] = Xi_z_to_repeat
                # And fill in the rest of the columns with straightforward repeateded data
                for i in 1:n_columns
                    if i != column_number # Don't overwrite the z coordinate
                        extra_padding[:,i] = data_array[mask_max_z .| mask_min_z,i]
                    end
                end
                # Now join it on to the data array
                data_array = vcat(data_array, extra_padding)

            end
        end
        
            
        coords = data_array[:,1:n_true_dims] # The first columns are the coordinates
        var_data = data_array[:,(n_true_dims+1):end] # The rest is the data
        
        for (ivar,var) in enumerate(original_var_names)
            
            values = var_data[:,ivar]
            interpolator = PyLinearNDInterpolator(coords, values)

            interp_data = interpolator(regular_coord_mesh...)

            new_var_loc = "timeseries/$var"*"_filtered_regrid/$iter"
            if haskey(file, new_var_loc)
                Base.delete!(file, new_var_loc) #incase we already tried to write this variable
            end
            file[new_var_loc] = interp_data
            
        end
    end
end


0.0%┣                                               ┫ 0/10 [00:00<00:00, -0s/it]
10.0%┣████▏                                     ┫ 1/10 [00:02<Inf:Inf, InfGs/it]
20.0%┣█████████▍                                     ┫ 2/10 [00:03<00:23, 3s/it]
30.0%┣██████████████                                 ┫ 3/10 [00:04<00:13, 2s/it]
40.0%┣██████████████████▉                            ┫ 4/10 [00:04<00:09, 1s/it]
50.0%┣███████████████████████▌                       ┫ 5/10 [00:05<00:06, 1s/it]
60.0%┣████████████████████████████▏                  ┫ 6/10 [00:06<00:05, 1s/it]
70.0%┣█████████████████████████████████              ┫ 7/10 [00:07<00:03, 1s/it]
80.0%┣█████████████████████████████████████▋         ┫ 8/10 [00:08<00:02, 1s/it]
90.0%┣██████████████████████████████████████████▎    ┫ 9/10 [00:08<00:01, 1s/it]
100.0%┣█████████████████████████████████████████████┫ 10/10 [00:09<00:00, 1s/it]
100.0%┣█████████████████████████████████████████████┫ 10/10 [00:09<00:00, 1s/it]


In [60]:
function remove_halos(data,grid)
    Hx = grid.Hx
    Hy = grid.Hy
    Hz = grid.Hz
    data = data[
                Hx != 0 ? (Hx+1:end-Hx) : (:),
                Hy != 0 ? (Hy+1:end-Hy) : (:),
                Hz != 0 ? (Hz+1:end-Hz) : (:)]
    return data
end

function _create_coords(grid)
    full_grid_size = (grid.Nx + 2*grid.Hx, grid.Ny+ 2*grid.Hy, grid.Nz+ + 2*grid.Hz)
    x = xnodes(grid, Center(), with_halos = true)
    y = ynodes(grid, Center(), with_halos = true)
    z = znodes(grid, Center(), with_halos = true)


    all_coords = (x=x,y=y,z=z)
    coord_dict = Dict()
    coord_dict["full_grid_size"] = full_grid_size
    for (coord_name, coord) in pairs(all_coords)
        if coord !== nothing
            coord_dict[String(coord_name)] = parent(coord)
            if coord_name == :x
                coord_dict["x_mesh"] = parent(coord) .+ zeros(full_grid_size)
            elseif coord_name == :y
                coord_dict["y_mesh"] = parent(coord)' .+ zeros(full_grid_size)
            elseif coord_name == :z
                coord_dict["z_mesh"] = reshape(parent(coord),1,1,length(parent(coord))) .+ zeros(full_grid_size)
            end
        end
    end
    return coord_dict
end

_create_coords (generic function with 1 method)

In [ ]:
file = jldopen(combined_output_filename,"r+")

Dict{Any, Any} with 4 entries:
  "x_mesh" => [-0.0613592 -0.0613592 … -0.0613592 -0.0613592; -0.0368155 -0.036…
  "x"      => -0.06135923151542565:0.02454369260617026:6.344544538695012
  "y_mesh" => [-0.0613592 -0.0368155 … 6.32 6.34454; -0.0613592 -0.0368155 … 6.…
  "y"      => -0.06135923151542565:0.02454369260617026:6.344544538695012

In [37]:
x = parent(xnodes(grid, Center(), with_halos = true))
full_grid_size = (grid.Nx + 2*grid.Hx, grid.Ny+ 2*grid.Hy, grid.Nz+ + 2*grid.Hz)
x .+ zeros(full_grid_size)


262×262×1 Array{Float64, 3}:
[:, :, 1] =
 -0.0613592  -0.0613592  -0.0613592  …  -0.0613592  -0.0613592  -0.0613592
 -0.0368155  -0.0368155  -0.0368155     -0.0368155  -0.0368155  -0.0368155
 -0.0122718  -0.0122718  -0.0122718     -0.0122718  -0.0122718  -0.0122718
  0.0122718   0.0122718   0.0122718      0.0122718   0.0122718   0.0122718
  0.0368155   0.0368155   0.0368155      0.0368155   0.0368155   0.0368155
  0.0613592   0.0613592   0.0613592  …   0.0613592   0.0613592   0.0613592
  0.0859029   0.0859029   0.0859029      0.0859029   0.0859029   0.0859029
  0.110447    0.110447    0.110447       0.110447    0.110447    0.110447
  0.13499     0.13499     0.13499        0.13499     0.13499     0.13499
  0.159534    0.159534    0.159534       0.159534    0.159534    0.159534
  0.184078    0.184078    0.184078   …   0.184078    0.184078    0.184078
  0.208621    0.208621    0.208621       0.208621    0.208621    0.208621
  0.233165    0.233165    0.233165       0.233165    0.233165    

In [25]:
x = xnodes(grid, Center(), with_halos = true)
y = ynodes(grid, Center(), with_halos = true)
z = znodes(grid, Center(), with_halos = true)
    
    
coords = (x=x,y=y,z=z)

(x = -0.06135923151542565:0.02454369260617026:6.344544538695012 with indices -2:259, y = -0.06135923151542565:0.02454369260617026:6.344544538695012 with indices -2:259, z = nothing)

In [27]:
for (key,value) in coords
    println(key)
    println(value)
end

-0.06135923151542565
-0.03681553890925539
-0.06135923151542565
-0.03681553890925539


LoadError: MethodError: no method matching iterate(::Nothing)

[0mClosest candidates are:
[0m  iterate([91m::LibGit2.GitRebase[39m)
[0m[90m   @[39m [35mLibGit2[39m [90m~/.julia/juliaup/julia-1.10.9+0.x64.linux.gnu/share/julia/stdlib/v1.10/LibGit2/src/[39m[90m[4mrebase.jl:48[24m[39m
[0m  iterate([91m::LibGit2.GitRebase[39m, [91m::Any[39m)
[0m[90m   @[39m [35mLibGit2[39m [90m~/.julia/juliaup/julia-1.10.9+0.x64.linux.gnu/share/julia/stdlib/v1.10/LibGit2/src/[39m[90m[4mrebase.jl:48[24m[39m
[0m  iterate([91m::IRTools.Inner.IR[39m)
[0m[90m   @[39m [36mIRTools[39m [90m~/.julia/packages/IRTools/v0mn8/src/ir/[39m[90m[4mir.jl:625[24m[39m
[0m  ...
